In [58]:
with open('log1128_143.txt', 'r') as f:
    all_logs = f.readlines() 

In [59]:
import re

dataset_list = []
lr_list = []
epochs_list = []
hidden_list = []
dropout_list = []
weight_decay_list = []
alpha_list = []
beta_list = []
gamma_list = []
norm_layers_list = []
orders_list = []
highest_train_list = []
highest_valid_list = []
final_train_list = []
final_test_list = []
delta_list = []

for logs in all_logs:
    if 'Namespace' in logs:
        dataset_list.extend(re.findall(r"dataset='(.*?)', delta", logs))
        delta_list.extend(re.findall(r"delta=(.*?), directed", logs))
        lr_list.extend(re.findall(r"lr=(.*?), method", logs))
        epochs_list.extend(re.findall(r"epochs=(.*?), gamma", logs))
        dropout_list.extend(re.findall(r"dropout=(.*?), epochs", logs))
        weight_decay_list.extend(re.findall(r"weight_decay=(.*?)\)", logs))
        alpha_list.extend(re.findall(r"alpha=(.*?), beta", logs))
        beta_list.extend(re.findall(r"beta=(.*?), cached", logs))
        orders_list.extend(re.findall(r"orders=(.*?), orders_func_id", logs))
        gamma_list.extend(re.findall(r"gamma=(.*?), gat_heads", logs))
        norm_layers_list.extend(re.findall(r"norm_layers=(.*?), num_layers", logs))
        hidden_list.extend(re.findall(r"hidden_channels=(.*?), hops", logs))
        
    if 'Highest Train' in logs and ',' in logs:
        highest_train_list.extend(re.findall(r"Highest Train: (.*?),", logs))
    
    if 'Highest Valid' in logs and ',' in logs:
        highest_valid_list.extend(re.findall(r"Highest Valid: (.*?),", logs))
        
    if 'Final Train' in logs and ',' in logs:
        final_train_list.extend(re.findall(r"Final Train: (.*?),", logs))
        
    if 'Final Test' in logs and ',' in logs:
        final_test_list.extend(re.findall(r"Final Test: (.*?),", logs))


# highest_train_list.append('0')
# highest_valid_list.append('0')
# final_train_list.append('0')
# final_test_list.append('0')
        

In [60]:
import pandas as pd
import os

d = {
    'dataset': dataset_list,
    'lr': lr_list,
    'epoch': epochs_list,
    'hidden': hidden_list,
    'dropout': dropout_list,
    'weight_decay': weight_decay_list,
    'alpha': alpha_list,
    'beta': beta_list,
    'gamma': gamma_list,
    'delta': delta_list,
    'norm_layers': norm_layers_list,
    'orders': orders_list,
    'high_train': highest_train_list,
    'high_valid': highest_valid_list,
    'final_train': final_train_list,
    'final_test': final_test_list
}

for k, v in d.items():
    print(k, len(v))

csv_name = 'results/large_dataset_tuning14.csv'
df = pd.DataFrame.from_dict(d)
df = df.sort_values(['final_test']).reset_index(drop=True)
# if os.path.exists(csv_name):
#     df.to_csv(csv_name, mode='a', header=False)
# else:
#     df.to_csv(csv_name)
df[-20:]

dataset 36
lr 36
epoch 36
hidden 36
dropout 36
weight_decay 36
alpha 36
beta 36
gamma 36
delta 36
norm_layers 36
orders 36
high_train 36
high_valid 36
final_train 36
final_test 36


,dataset,lr,epoch,hidden,dropout,weight_decay,alpha,beta,gamma,delta,norm_layers,orders,high_train,high_valid,final_train,final_test
16,snap-patents,0.005,100,64,0.6,0.01,0.0,1.0,0.9,0.3,1,3,89.00,61.48,88.19,61.48
17,snap-patents,0.005,100,64,0.6,0.1,0.0,1.0,0.8,0.3,1,3,88.97,61.51,88.05,61.51
18,snap-patents,0.005,100,64,0.6,0.01,0.0,1.0,0.9,0.5,1,3,88.99,61.52,88.05,61.51
19,snap-patents,0.005,100,64,0.8,0.01,0.0,1.0,0.8,0.5,1,3,88.60,61.63,88.48,61.65
20,snap-patents,0.005,100,64,0.8,0.1,0.0,1.0,0.9,0.5,1,3,88.66,61.68,88.40,61.66
21,snap-patents,0.005,100,64,0.8,0.1,0.0,1.0,0.9,0.4,1,3,88.67,61.70,88.45,61.68
22,snap-patents,0.005,100,64,0.8,0.01,0.0,1.0,0.8,0.3,1,3,88.70,61.74,88.44,61.71
23,snap-patents,0.005,100,64,0.8,0.1,0.0,1.0,0.7,0.3,1,3,88.69,61.74,88.37,61.73
24,snap-patents,0.005,100,64,0.8,0.1,0.0,1.0,0.8,0.5,1,3,88.67,61.78,88.13,61.75
25,snap-patents,0.005,100,64,0.8,0.1,0.0,1.0,0.7,0.5,1,3,88.55,61.76,88.20,61.76
